In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Dog&Cat  SVM

---

### Team members:
* Zhichun Wang ID 19024898
* Ziyi Wang   ID 18042783
* Youzhi Lei  ID 19039281

## Content
* Using SVM to classify some kinds of vehicles & animals

---


#### Information about datasets
Labeled vehicles and animals from:
    Images of various types of vehicles and animals http://www.cs.toronto.edu/%7Ekriz/cifar.html 

In [2]:
#!pip install opencv-python

In [3]:
#!pip install tensorflow

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import patsy
import os
from PIL import Image 
import cv2
%matplotlib inline

In [5]:
# import the necessary packages
import sklearn
print(sklearn.__version__)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import cv2
import os

0.20.2


In [6]:
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    # handle normalizing the histogram if we are using OpenCV 2.4.X
    # if imutils.is_cv2():
    #    hist = cv2.normalize(hist)
    #print("line2")
    # otherwise, perform "in place" normalization in OpenCV 3
    #else:
    cv2.normalize(hist, hist)
    
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [7]:
from pylab import rcParams

sns.set(style="ticks")
#sns.set_style("whitegrid")
rcParams['figure.dpi'] = 150
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'StixGeneral'
rcParams['figure.figsize'] = 10,8
rcParams['font.size'] = 14
rcParams['axes.labelsize'] = 'large'
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

pd.options.display.max_columns = 50

In [8]:
import os
cwd = os.getcwd()
print(cwd)
#Please put the data to current location, Thanks 

/home/dp1/Documents/fred/codes/machinelearning2019/cifar10


In [9]:
#load the dataset 
#test one image and show 
DATADIR = "../cifar-10-batches-py"
DATADIR = os.path.join(cwd, DATADIR)
print(DATADIR)


/home/dp1/Documents/fred/codes/machinelearning2019/cifar10/../cifar-10-batches-py


In [12]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Choose one image randomly, just to display it
batch = unpickle(DATADIR+"/data_batch_1")

# For more detail on the dataset files, see http://www.cs.toronto.edu/%7Ekriz/cifar.html 
IMG_SIZE = 32 # each image in the dataset is 32*32, color
img_array = batch[b'data'][0].reshape(3, 1024)
ziplist = list(zip(img_array[2], img_array[1], img_array[0]))
newarray = np.array(ziplist).reshape(IMG_SIZE, IMG_SIZE, 3)

#plt.imshow(newarray)
#plt.show

img_array = cv2.cvtColor(newarray, cv2.COLOR_BGR2GRAY)
#plt.imshow(img_array, cmap='gray')
#plt.show

metafile = unpickle(DATADIR+"/batches.meta") 
print("This image is a", metafile[b'label_names'][batch[b'labels'][0]], ". Its label is", batch[b'labels'][0])

CATEGORIES = []
for i in range(len(metafile[b'label_names'])):
    CATEGORIES.append(metafile[b'label_names'][i])

print(len(metafile[b'label_names']), "categories:", CATEGORIES)

This image is a b'frog' . Its label is 6
10 categories: [b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']


In [13]:
rawImages = []
features = []
labels = []

def create_training_data():    
    for batchfile in os.listdir(DATADIR):        
        if (batchfile.startswith("data_batch_") or batchfile == 'test_batch'):                
            batch = unpickle(os.path.join(DATADIR, batchfile))
            print(len(batch[b'data']), "images are in", batchfile)
            for i in range(len(batch[b'data'])): # There should be 10000 images in each batch                    
                # 1024 red values, then 1024 green values, then 1024 blue values
                img_array = batch[b'data'][i].reshape(3, 1024) 
                #print("shape is", img_array.shape)
                # Note that OpenCV needs BGR instead of RGB
                ziplist = list(zip(img_array[2], img_array[1], img_array[0])) 
                img_array = np.array(ziplist).reshape(IMG_SIZE, IMG_SIZE, 3)
                pixels = image_to_feature_vector(img_array)    
                hist = extract_color_histogram(img_array)       
                label = batch[b'labels'][i]
                
                rawImages.append(pixels)
                features.append(hist)
                labels.append(label)                    
create_training_data()


10000 images are in data_batch_5
10000 images are in data_batch_2
10000 images are in data_batch_3
10000 images are in data_batch_4
10000 images are in test_batch
10000 images are in data_batch_1


In [14]:
print(len(rawImages))
print(len(features))
print(len(labels))


60000
60000
60000


In [15]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(features.nbytes / (1024 * 1000.0)))
print(labels)


[INFO] pixels matrix: 180.00MB
[INFO] features matrix: 120.00MB
[1 8 5 ... 1 1 5]


In [16]:
from sklearn import preprocessing
#features_scaled = preprocessing.scale(features)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled  = scaler.fit(features)

In [17]:
# partition the data into training and testing splits, using 85%
# of the data for training and the remaining 15% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(rawImages, labels, test_size=0.15, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(features, labels, test_size=0.15, random_state=42)
#(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(features_scaled, labels, test_size=0.15, random_state=42)

In [18]:
print(trainRI.shape)

(51000, 3072)


In [ ]:
# k-NN
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1)
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
#print("[INFO] k-NN classifier: k=%d" % args["neighbors"])
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))




[INFO] evaluating raw pixel accuracy...


In [ ]:
#neural network
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = MLPClassifier(hidden_layer_sizes=(32,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=.1)
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] neural network raw pixel accuracy: {:.2f}%".format(acc * 100))

In [ ]:
#neural network
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = MLPClassifier(hidden_layer_sizes=(32,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=.1)
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] neural network histogram accuracy: {:.2f}%".format(acc * 100))


In [19]:
%%time
#SVC
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = SVC(max_iter=1000,class_weight='balanced')
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] SVM-SVC raw pixel accuracy: {:.2f}%".format(acc * 100))




[INFO] evaluating raw pixel accuracy...


/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] SVM-SVC raw pixel accuracy: 10.64%
CPU times: user 48min 59s, sys: 699 ms, total: 49min
Wall time: 48min 58s


In [20]:
%%time
#SVC
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = SVC(max_iter=1000,class_weight='balanced')
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] SVM-SVC histogram accuracy: {:.8f}%".format(acc * 100))



[INFO] evaluating histogram accuracy...


/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] SVM-SVC histogram accuracy: 10.43333333%
CPU times: user 7min 38s, sys: 116 ms, total: 7min 38s
Wall time: 7min 37s
